In [1]:
using MambaModels

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;
line[:x] = convert(Array{Float64,1}, df2[:weight_c]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

352×2 Array{Float64,2}:
 1.0    2.83512
 1.0   -8.50468
 1.0  -13.1256 
 1.0    8.05143
 1.0   -3.71361
 1.0   18.0021 
 1.0   -6.74701
 1.0   10.4895 
 1.0  -10.1206 
 1.0    9.49725
 ⋮             
 1.0    2.89182
 1.0   -5.58468
 1.0   -3.94041
 1.0   -4.16721
 1.0    2.04133
 1.0  -10.7443 
 1.0    7.17259
 1.0    9.07201
 1.0    7.54114

Model Specification

In [3]:
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)

Object of type "Model"
-------------------------------------------------------------------------------
beta:
A monitored node of type "0-element ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
y:
An unmonitored node of type "0-element ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "ScalarStochastic"
NaN


Initial Values

In [4]:
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

3-element Array{Dict{Symbol,Any},1}:
 Dict(:beta=>[92.0371, -0.276722],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>20.6571)
 Dict(:beta=>[154.34, 6.38961],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>6.28509)   
 Dict(:beta=>[334.046, 0.346906],:y=>[151.765, 139.7, 136.525, 156.845, 145.415, 163.83, 149.225, 168.91, 147.955, 165.1  …  156.21, 160.655, 146.05, 156.21, 152.4, 162.56, 142.875, 162.56, 156.21, 158.75],:s2=>36.0937) 

Tuning Parameters

In [5]:
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

0.1

Define sampling scheme

In [6]:
scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

Object of type "Model"
-------------------------------------------------------------------------------
beta:
A monitored node of type "0-element ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
y:
An unmonitored node of type "0-element ArrayStochastic{1}"
Float64[]
-------------------------------------------------------------------------------
s2:
A monitored node of type "ScalarStochastic"
NaN


MCMC Simulation

In [7]:
chn = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:00:07 of 0:00:07 remaining]
Chain 1:  10% [0:00:03 of 0:00:03 remaining]
Chain 1:  20% [0:00:03 of 0:00:03 remaining]
Chain 1:  30% [0:00:02 of 0:00:03 remaining]
Chain 1:  40% [0:00:02 of 0:00:03 remaining]
Chain 1:  50% [0:00:02 of 0:00:03 remaining]
Chain 1:  60% [0:00:01 of 0:00:03 remaining]
Chain 1:  70% [0:00:01 of 0:00:03 remaining]
Chain 1:  80% [0:00:01 of 0:00:03 remaining]
Chain 1:  90% [0:00:00 of 0:00:03 remaining]
Chain 1: 100% [0:00:00 of 0:00:03 remaining]

Chain 2:   0% [0:00:04 of 0:00:04 remaining]
Chain 2:  10% [0:00:03 of 0:00:03 remaining]
Chain 2:  20% [0:00:03 of 0:00:03 remaining]
Chain 2:  30% [0:00:02 of 0:00:03 remaining]
Chain 2:  40% [0:00:02 of 0:00:03 remaining]
Chain 2:  50% [0:00:02 of 0:00:03 remaining]
Chain 2:  60% [0:00:01 of 0:00:03 remaining]
Chain 2:  70% [0:00:01 of 0:00:03 remaining]
Chain 2:  80% [0:00:01 of 0:00:03 remaining]
Chain 2:  90% [0:00:00 of 0:00:03 remaining]
Cha

Object of type "ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[24.9151 154.223 0.910975; 28.9412 154.223 0.910975; … ; 30.9348 154.868 0.81032; 31.3837 154.82 0.905846]

[27.7931 154.833 0.895606; 28.8957 154.433 0.879357; … ; 24.4363 154.221 0.885701; 26.1145 154.894 0.921221]

[27.1051 154.187 0.900425; 22.1019 154.152 0.869584; … ; 25.8846 154.606 0.847487; 27.5717 154.686 0.836989]

Show draws summary

In [8]:
describe(chn)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE        MCSE        ESS  
     s2  26.1808285 1.987645818 0.01209642723 0.02096218060 8990.940
beta[1] 154.6007456 0.268759623 0.00163561898 0.00368432994 5321.218
beta[2]   0.9050737 0.042271503 0.00025725617 0.00030863195 9000.000

Quantiles:
            2.5%       25.0%       50.0%        75.0%        97.5%   
     s2  22.5708796  24.8109076  26.0765976  27.45009116  30.40000216
beta[1] 154.0763392 154.4204393 154.5978358 154.78141263 155.13160391
beta[2]   0.8209637   0.8773362   0.9050824   0.93292768   0.98888507



End of `m4.1m.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*